<a href="https://colab.research.google.com/github/oreomcflurryyy/statistical-learning-deeplearning/blob/main/deeplearning_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ISLP
!pip install torchinfo
!pip install pytorch-lightning
!pip install torchmetrics

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 8.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=f13b3764816443699193ab53a0c29d8607ca44353a4794a993882f4ad5f9179c
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  Attempting 

In [3]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)

In [4]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

In [5]:
from torchmetrics import (MeanAbsoluteError,
                          R2Score)
from torchinfo import summary

In [6]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

In [7]:
from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)

INFO:lightning_fabric.utilities.seed:Seed set to 0


In [8]:
from torchvision.io import read_image
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50,
                                ResNet50_Weights)
from torchvision.transforms import (Resize,
                                    Normalize,
                                    CenterCrop,
                                    ToTensor)

In [9]:
from ISLP.torch import (SimpleDataModule,
                        SimpleModule,
                        ErrorTracker,
                        rec_num_workers)

In [10]:
from ISLP.torch.imdb import (load_lookup,
                             load_tensor,
                             load_sparse,
                             load_sequential)

In [25]:
from glob import glob
import json
from IPython.display import display, HTML
from tabulate import tabulate

From your collection of personal photographs, pick 10 images of an-
imals (such as dogs, cats, birds, farm animals, etc.). If the subject
does not occupy a reasonable part of the image, then crop the image.
Now use a pretrained image classification CNN as in Lab 10.9.4 to
predict the class of each of your images, and report the probabilities
for the top five predicted classes for each image.

In [15]:
resize = Resize((232,232), antialias=True)
crop = CenterCrop(224)
normalize = Normalize([0.485,0.456,0.406],
                      [0.229,0.224,0.225])
imgfiles = sorted([f for f in glob('book_images/*')])
imgs = torch.stack([torch.div(crop(resize(read_image(f))), 255)
                    for f in imgfiles])
imgs = normalize(imgs)
imgs.size()

torch.Size([9, 3, 224, 224])

In [16]:
resnet_model = resnet50(weights=ResNet50_Weights.DEFAULT)
summary(resnet_model,
        input_data=imgs,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 125MB/s]


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
ResNet                                   [9, 3, 224, 224]          [9, 1000]                 --
├─Conv2d: 1-1                            [9, 3, 224, 224]          [9, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [9, 64, 112, 112]         [9, 64, 112, 112]         128
├─ReLU: 1-3                              [9, 64, 112, 112]         [9, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [9, 64, 112, 112]         [9, 64, 56, 56]           --
├─Sequential: 1-5                        [9, 64, 56, 56]           [9, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [9, 64, 56, 56]           [9, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [9, 64, 56, 56]           [9, 64, 56, 56]           4,096
│    │    └─BatchNorm2d: 3-2             [9, 64, 56, 56]           [9, 64, 56, 56]           128
│    │    └─ReLU: 3-3      

In [17]:
resnet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
img_preds = resnet_model(imgs)

In [19]:
img_probs = np.exp(np.asarray(img_preds.detach()))
img_probs /= img_probs.sum(1)[:,None]

In [20]:
labs = json.load(open('imagenet_class_index.json'))
class_labels = pd.DataFrame([(int(k), v[1]) for k, v in
                           labs.items()],
                           columns=['idx', 'label'])
class_labels = class_labels.set_index('idx')
class_labels = class_labels.sort_index()

In [26]:
for i, imgfile in enumerate(imgfiles):
    img_df = class_labels.copy()
    img_df['prob'] = img_probs[i]
    img_df = img_df.sort_values(by='prob', ascending=False)[:5]
    print(f'Image: {imgfile}')

    table = tabulate(img_df.reset_index().drop(columns=['idx']), headers='keys', tablefmt='html')
    display(HTML(table))

Image: book_images/Cape_Weaver.jpg


,label,prob
0,jacamar,0.297499
1,macaw,0.0681069
2,lorikeet,0.0511044
3,bee_eater,0.0444302
4,bulbul,0.0291235


Image: book_images/Flamingo.jpg


,label,prob
0,flamingo,0.609514
1,spoonbill,0.013586
2,American_egret,0.0021321
3,pelican,0.00136544
4,crane,0.00126401


Image: book_images/Hawk_Fountain.jpg


...

,label,prob
0,ox,0.630883
1,oxcart,0.022181
2,water_buffalo,0.00725485
3,ram,0.00473257
4,sorrel,0.00427067


Image: book_images/german_shepherd.jpeg


,label,prob
0,German_shepherd,0.358803
1,Leonberg,0.00688311
2,patas,0.00413909
3,briard,0.00383933
4,malinois,0.00377497


Image: book_images/king_cobra.jpg


,label,prob
0,Indian_cobra,0.479893
1,king_snake,0.00448407
2,hognose_snake,0.00423576
3,night_snake,0.00403787
4,horned_viper,0.00294265
